### Imports

In [1]:
import sqlite3
import pyterrier as pt
import os
from pathlib import Path
from bs4 import BeautifulSoup  # To clean the html
import warnings
warnings.filterwarnings("ignore")

if not pt.started():
    pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
C:\Users\ajsko\AppData\Local\Temp\ipykernel_43712\2994591996.py:10: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


### SQLite DB for the Scraped Data

In [2]:
conn = sqlite3.connect("../webscraper/crawled.db")  # Path to database

c = conn.cursor()

In [8]:
def extract_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text().replace('\n', '')

In [9]:
c.execute("SELECT url, html FROM paper")
documents = []
for url, html in c.fetchall():
    documents.append({"docno": url, "text": extract_text(html)})

In [10]:
documents[0]

{'docno': 'https://deepmind.google/research/publications/101479',
 'text': 'Machine Unlearning Doesn’t Do What You Think: Lessons for Generative AI Policy, Research, and Practice - Google DeepMind      Jump to Content    GoogleDeepMindSearch...SearchCloseGoogleDeepMind          About                                        Learn about Google DeepMind                                       — Our mission is to build AI responsibly to benefit humanity                    Responsibility & Safety                                       — We want AI to benefit the world, so we must be thoughtful about how it’s built and used                    Education                                       — Our vision is to help make the AI ecosystem more representative of society                    Careers                                       — Many disciplines, one common goal                    Research                                        View Research                                       — We work on s

### Indexing

In [ ]:
index_path = Path("../webscraper/index")
if os.path.exists(index_path):
    index = pt.IndexFactory.of(index_path)
else:
    indexer = pt.IterDictIndexer("../webscraper/index", meta={"docno": 20, "filename": 1024, "title": 1024}, meta_tags={"title":"title"})
    index_ref = indexer.index(documents)
    index = pt.IndexFactory.of(index_ref)

In [ ]:
print(index.getCollectionStatistics().toString())

In [ ]:
print("Available keys in MetaIndex:", index.getMetaIndex().getKeys())

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25.transform("LLM")